In [ ]:
import os
import sys
sys.path.insert(0, "../../")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from src.text.epu import EPU

In [ ]:
parent_dirs = sys.path[0] + "data/text/"
country_dirs = [
    parent_dirs + country for country in os.listdir(parent_dirs)
    if country != ".DS_Store" and "marshall_islands" not in country
]
output_dir = sys.path[0] + "outputs/text/"

In [ ]:
for country in country_dirs:
    country_name = country.split("/")[-1]
    news_dirs = [
        f"{country}/{file}" for file in os.listdir(country) if "news" in file
    ]
    
    e = EPU(news_dirs, cutoff="2020-12-31")
    e.get_epu_category(subset_condition="date >= '2015-01-01' and date < '2024-01-01'")
    e.get_count_stats()
    e.calculate_epu_score()
    
    epu_stats = e.epu_stats
    fig, ax = plt.subplots(figsize=(8, 6))
    epu_stats.plot(x="date", y="epu_weighted", color="blue", ax=ax)
    epu_stats.plot(x='date', y="epu_unweighted", color="lightgray", ax=ax)
    
    title = " ".join(n[0].upper() + n[1:] for n in country_name.split("_"))
    ax.set_title(f"{title}")
    ax.set_xlabel("Date")
    
    epu_stats.to_csv(output_dir + f"{country_name}/{country_name}_epu.csv", encoding="utf-8")